## Terraforming Mars Cost-Benefit Analysis
<br>
Terraforming Mars is a competitive board game developed by Jacob Fryxelius and published by FryxGames in 2016. Players take on the role of corporations tasked with making Mars habitable through the raising of the oxygen level, temperature, and ocean coverage, as well as sundry other projects to progress humanity.

Currently sitting at #4 on BoardGameGeek's ranked board game list, Terraforming Mars has found broad popularity among strategy board gamers in large part due to its variety and complexity. With 208 unique cards and 12 distinct corporations, players are put into an endless assortment of scenarios. This incredible depth means that players must often intuit their next move rather than deduce it from values and probabilities.

In a bid to add more deduction back into the equation, this analysis aims to take every card in Terraforming Mars and boil it down to a single value in credits (the game's main resource) for simple value-comparison.

Let's get started.

In [1]:
import numpy as np
from statistics import median
import pandas as pd
import seaborn as sns
import csv
import ast
%run ./TM_lib.ipynb # allows use of functions in TM_lib.ipynb

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent
5,Aerobraked Ammonia Asteroid,base,event,26,NaN,"space, event",NaN,NaN,"2 microbe on other card, 3 HEAT, 1 PLANT",NaN,NaN,NaN,NaN
13,Artificial Lake,base,automated,15,NaN,building,-6°C,1,1 ocean on site not reserved for ocean,NaN,NaN,NaN,NaN
27,Business Contacts,corporate,event,7,NaN,"earth, event",NaN,NaN,"4 draw, then discard 2 of them",NaN,NaN,NaN,NaN
51,Ecological Zone,base,active,12,NaN,"animal, plant",player’s 1 greenery tile,1 per 2 animal on card,special tile adjacent to any greenery,"when playing an animal or plant tag, 1 animal to card",NaN,NaN,NaN
55,Eos Chasma National Park,base,automated,16,NaN,"plant, building",-12°C,1,"1 animal on other card, 3 plant, 2 CREDIT",NaN,NaN,NaN,NaN
65,Ganymede Colony,base,automated,20,NaN,"jovian, space, city",NaN,1 per player jovian tag,1 city on reserved site,NaN,NaN,NaN,NaN
87,Imported Hydrogen,base,event,16,NaN,"earth, space, event",NaN,NaN,"3 plant or 3 microbe on other card or 2 animal on other card, 1 ocean",NaN,NaN,NaN,NaN
88,Imported Nitrogen,base,event,23,NaN,"earth, space, event",NaN,NaN,"1 TR, 4 plant, 3 microbe on other card, 2 animal on other card",NaN,NaN,NaN,NaN
89,Indentured Workers,corporate,event,0,NaN,event,NaN,-1,the next card played by player costs 8 credit fewer,NaN,NaN,NaN,NaN
90,Industrial Center,corporate,active,4,NaN,building,NaN,NaN,special tile adjacent to any city,NaN,7 credit,1 STEEL,NaN


NameError: name 'vars_scan' is not defined

The base_cards database, populated from a hand-entered csv, follows a couple of conventions:
* When a resource is written in uppercase it refers to that resource's production, whereas when written in lowercase it refers to the resource iself.
* All nouns are written in the singular in order to simplify pandas referencing.

The all_games database is a collection of stats from over 12,000 individually logged games downloaded from Simeon Simeonov's excellent [Terraforming Mars website](https://ssimeonoff.github.io/). The base_games database contains only the matches from all_games which were played with the full base version of the game.

In [2]:
# create databases
base_cards = pd.read_csv('TM_base_project_list.csv')
all_games = pd.read_csv('games_list.csv')
base_games = all_games[(all_games.expansions == "['CORPORATE']") & (all_games.map == "THARSIS")]

First, an average number of generations for each player count must be derived from base_games. These will be used to determine the income which will be generated over the remainder of the game for a given card after the generation during which it is played. They'll also be used to split the game into four quarters to illustrate how a card's value changes over time.

In [3]:
# determine generation averages per player count
lastgen2p = int(round(base_games[base_games.players == 2].generations.mean()))
lastgen3p = int(round(base_games[base_games.players == 3].generations.mean()))
lastgen4p = int(round(base_games[base_games.players == 4].generations.mean()))
lastgen5p = int(round(base_games[base_games.players == 5].generations.mean()))

lastgens = {
            'lastgen2p': int(round(base_games[base_games.players == 2].generations.mean())),
            'lastgen3p': int(round(base_games[base_games.players == 3].generations.mean())),
            'lastgen4p': int(round(base_games[base_games.players == 4].generations.mean())),
            'lastgen5p': int(round(base_games[base_games.players == 5].generations.mean()))
           }

print(f'''
Average length of game in generations:
2p: {lastgen2p}
3p: {lastgen3p}
4p: {lastgen4p}
5p: {lastgen5p}
''')

# average game length split into quarters with the median generation taken from each quarter
# X players, quarter N, median generation
gen_quarters = {
               '2pq1gen':round(lastgen2p/4/2),
               '2pq2gen':round(lastgen2p/4/2 + lastgen2p/4),
               '2pq3gen':round(lastgen2p/4/2 + lastgen2p/4*2),
               '2pq4gen':round(lastgen2p/4/2 + lastgen2p/4*3),
               '3pq1gen':round(lastgen3p/4/2),
               '3pq2gen':round(lastgen3p/4/2 + lastgen3p/4),
               '3pq3gen':round(lastgen3p/4/2 + lastgen3p/4*2),
               '3pq4gen':round(lastgen3p/4/2 + lastgen3p/4*3),
               '4pq1gen':round(lastgen4p/4/2),
               '4pq2gen':round(lastgen4p/4/2 + lastgen4p/4),
               '4pq3gen':round(lastgen4p/4/2 + lastgen4p/4*2),
               '4pq4gen':round(lastgen4p/4/2 + lastgen4p/4*3),
               '5pq1gen':round(lastgen5p/4/2),
               '5pq2gen':round(lastgen5p/4/2 + lastgen5p/4),
               '5pq3gen':round(lastgen5p/4/2 + lastgen5p/4*2),
               '5pq4gen':round(lastgen5p/4/2 + lastgen5p/4*3)
              }
keys2pgen = ['2pq1gen', '2pq2gen', '2pq3gen', '2pq4gen']
keys3pgen = ['3pq1gen', '3pq2gen', '3pq3gen', '3pq4gen']
keys4pgen = ['4pq1gen', '4pq2gen', '4pq3gen', '4pq4gen']
keys5pgen = ['5pq1gen', '5pq2gen', '5pq3gen', '5pq4gen']

print(f'''
Generation quarters:
2p: {str([gen_quarters.get(key) for key in keys2pgen]).strip('[]')}
3p: {str([gen_quarters.get(key) for key in keys3pgen]).strip('[]')}
4p: {str([gen_quarters.get(key) for key in keys4pgen]).strip('[]')}
5p: {str([gen_quarters.get(key) for key in keys5pgen]).strip('[]')}
''')



Average length of game in generations:
2p: 13
3p: 11
4p: 10
5p: 10


Generation quarters:
2p: 2, 5, 8, 11
3p: 1, 4, 7, 10
4p: 1, 4, 6, 9
5p: 1, 4, 6, 9



Next, each resource in the game must be converted to a number of credits. The [standard projects](https://3.bp.blogspot.com/-fs7vQZGRFs4/XAjAYOSBspI/AAAAAAAAFSA/kInbyuaLoLc0PANwJeFDpJditT5Ims3_wCLcBGAs/s1600/terraforming-mars-standard-projects-1.jpg) are set as listed on the game board, and steel and titanium as listed on the [player boards](https://cf.geekdo-images.com/medium/img/qa7YwBE6pc-ZR0relFRrzUAlQXw=/fit-in/500x500/filters:no_upscale()/pic2891980.jpg).  

Plants and heat are set by dividing 8, the number of plants/heat needed to equal a temp/greenery (also illustrated on the player boards), into the credit values for the temp and greenery standard projects.  

For energy, it was necessary to borrow some information from one of the expansions: _Colonies_. In order to trade with a colony, players must spend either 9 credits, 3 titanium, or 3 energy. This suggests that 1 energy = 1 titanium = 3 credits.  

Drawing a card is valued at 4 credits, combining the price to buy a card (3 credits) with the 1 credit gained from discarding it using the "Sell patents" standard project.

In [4]:
# variable definitions

income = 0
vp = 0
TR = vp + income

# standard projects
temp = 14
ocean = 18
greenery = 23
def city(income):
    return 25 - income
oxygen = greenery / 2

# resources
credit = 1
steel = 2
titanium = 3
plant = greenery / 8
heat = temp / 8
energy = 3
draw = 4

# production
def CREDIT(income):
    return income
def STEEL(income):
    return 2 * income
def TITANIUM(income):
    return 3 * income
def PLANT(income):
    return (greenery/8) * income
def ENERGY(income):
    return 3 * income
def HEAT(income):
    return (temp/8) * income

vars_dict_static = {'temp':temp, 'ocean':ocean, 'greenery':greenery, 'credit':credit, 'steel':steel,
                     'titanium':titanium, 'plant':plant, 'heat':heat, 'energy':energy, 'draw':draw}

vars_dict_variable = {'city':city, 'CREDIT':CREDIT, 'STEEL':STEEL, 'TITANIUM':TITANIUM,
                      'PLANT':PLANT, 'ENERGY':ENERGY, 'HEAT':HEAT}

vars_dict_resource = {}

def vars_scan(benefit_list):
    for var in vars_dict_static:
        if var == benefit_list[1]:
            benefit_value = int(benefit_list[0]) * vars_dict_static[var]
    for var in vars_dict_variable:
        if var == benefit_list[1]:
            benefit_value = int(benefit_list[0]) * vars_dict_variable[var](income)
    for var in vars_dict_resource:
        if var == benefit_list[1]:
            benefit_value = int(benefit_list[0]) * vars_dict_resource[var](generation, player_count)
    if benefit_list[1] == 'TR':
        benefit_value = int(benefit_list[0]) * TR
    return benefit_value

In [5]:
# determine number of remaining generations
def remgen(generation, player_count):
    global income
    for players in lastgens:
        if str(player_count) in players:
            lastgen = lastgens[players]
    income = lastgen - generation
    return income

Victory points (VP) must also be converted to credits. To do this, the three terraforming standard projects were used. Since they confer a terraforming rating (TR) and not a victory point, the extra income from the TR must be subtracted from the cost of the project. This means that similar to the cards themselves, the value of a VP changes over time.

To come to a single value for a given generation and player count, the "points per credit" are determined for each of the three standard terraforming projects and then the average is taken. Note that greeneries are worth two points (one for the TR from raising the oxygen, another from the greenery tile itself) and so are calculated accordingly.

In [6]:
# define credits per victory point
def credits_per_vp(generation, player_count):
    income = remgen(generation, player_count)
    temp_ppc = 1/(temp - income) # ppc = points per credit
    ocean_ppc = 1/(ocean - income)
    greenery_ppc = 2/(greenery - income)
    ppc = (temp_ppc + ocean_ppc + greenery_ppc)/4
    vp = 1/ppc
    return vp

# numbers of credits per victory point for each player count and median generation
# X players, quarter N, credits per point
gen_credits_per_vp = {'2pq1cpp':credits_per_vp(2, 2),
                      '2pq2cpp':credits_per_vp(5, 2),
                      '2pq3cpp':credits_per_vp(8, 2),
                      '2pq4cpp':credits_per_vp(11, 2),
                      '3pq1cpp':credits_per_vp(1, 3),
                      '3pq2cpp':credits_per_vp(4, 3),
                      '3pq3cpp':credits_per_vp(7, 3),
                      '3pq4cpp':credits_per_vp(10, 3),
                      '4pq1cpp':credits_per_vp(1, 4),
                      '4pq2cpp':credits_per_vp(4, 4),
                      '4pq3cpp':credits_per_vp(6, 4),
                      '4pq4cpp':credits_per_vp(9, 4),
                      '5pq1cpp':credits_per_vp(1, 5),
                      '5pq2cpp':credits_per_vp(4, 5),
                      '5pq3cpp':credits_per_vp(6, 5),
                      '5pq4cpp':credits_per_vp(9, 5)
                     }

keys2pcpp = ['2pq1cpp', '2pq2cpp', '2pq3cpp', '2pq4cpp']
keys3pcpp = ['3pq1cpp', '3pq2cpp', '3pq3cpp', '3pq4cpp']
keys4pcpp = ['4pq1cpp', '4pq2cpp', '4pq3cpp', '4pq4cpp']
keys5pcpp = ['5pq1cpp', '5pq2cpp', '5pq3cpp', '5pq4cpp']

print(f'''
Generation credits per point per quarter:
   q1, q2, q3, q4
2p: {str([round(gen_credits_per_vp.get(key)) for key in keys2pcpp]).strip('[]')}
3p: {str([round(gen_credits_per_vp.get(key)) for key in keys3pcpp]).strip('[]')}
4p: {str([round(gen_credits_per_vp.get(key)) for key in keys4pcpp]).strip('[]')}
5p: {str([round(gen_credits_per_vp.get(key)) for key in keys5pcpp]).strip('[]')}
''')


Generation credits per point per quarter:
   q1, q2, q3, q4
2p: 6, 10, 13, 17
3p: 8, 11, 14, 18
4p: 9, 12, 14, 18
5p: 9, 12, 14, 18



In [7]:
#run above functions and determine credits per TR
def initialize(generation, player_count):
    income = remgen(generation, player_count)
    vp = credits_per_vp(generation, player_count)
    TR = vp + income
    return income, vp, TR

In [8]:
# chance to draw each tag

# Earth
earth_tags = base_cards[base_cards.Tags.str.contains('earth', na=False)]
chance_earth = len(earth_tags) / len(base_cards)
# Science
science_tags = base_cards[base_cards.Tags.str.contains('science', na=False)]
chance_science = len(science_tags) / len(base_cards)
# Plant
plant_tags = base_cards[base_cards.Tags.str.contains('plant', na=False)]
chance_plant = len(plant_tags) / len(base_cards)
# Microbe
microbe_tags = base_cards[base_cards.Tags.str.contains('microbe', na=False)]
chance_microbe = len(microbe_tags) / len(base_cards)
# Animal
animal_tags = base_cards[base_cards.Tags.str.contains('animal', na=False)]
chance_animal = len(animal_tags) / len(base_cards)
# Space
space_tags = base_cards[base_cards.Tags.str.contains('space', na=False)]
chance_space = len(space_tags) / len(base_cards)
# Event
event_tags = base_cards[base_cards.Tags.str.contains('event', na=False)]
chance_event = len(event_tags) / len(base_cards)
# Building
building_tags = base_cards[base_cards.Tags.str.contains('building', na=False)]
chance_building = len(building_tags) / len(base_cards)
# Jovian
jovian_tags = base_cards[base_cards.Tags.str.contains('jovian', na=False)]
chance_jovian = len(jovian_tags) / len(base_cards)
# Power
power_tags = base_cards[base_cards.Tags.str.contains('power', na=False)]
chance_power = len(power_tags) / len(base_cards)
# City
city_tags = base_cards[base_cards.Tags.str.contains('city', na=False)]
chance_city = len(city_tags) / len(base_cards)

tags_dict = {'earth':chance_earth, 'science':chance_science, 'plant':chance_plant,
             'microbe':chance_microbe, 'animal':chance_animal, 'space':chance_space,
             'event':chance_event, 'building':chance_building, 'jovian':chance_jovian,
             'power':chance_power, 'city':chance_city}

#### Tile placement

Placing a tile on an area with resources allows the player to take those resources, adding to the value of the tile. To determine the average value added from these map resources, the total sum of credits gained from all of the resources on the map are divided by the total number of areas. This is done separately for ocean areas and non-ocean areas. The two off-world areas and the Noctis City area have been disregarded, since they cannot be placed upon except for unique circumstances.

<img src="https://4.bp.blogspot.com/-KVleOqhdyg0/XBxty1rpALI/AAAAAAAAFac/iJsDNEW0SDw6gTK2aHIS0oLc9px5XvzjQCEwYBhgL/s1600/TM%2BOriginal-Mars%2BOnly.jpg" alt="TM map" width="500"/>

In [9]:
#determine added value from gained resources when placing a tile

tile_land = (
            (2*steel * 3 +
            draw * 3 +
            steel * 3 +
            plant + titanium +
            plant * 10 +
            2*plant * 7 +
            titanium)
            / 48)

tile_ocean = (
             (2*steel +
             draw +
             plant * 3 +
             2*plant * 4 +
             2*titanium)
             / 12)

print(f'Placing a land tile is worth {str(round(tile_land, 3))} credits.')
print(f'Placing an ocean tile is worth {str(round(tile_ocean, 3))} credits.')

Placing a land tile is worth 2.247 credits.
Placing an ocean tile is worth 3.802 credits.


#### Microbes  
The following cards are taken into account when calculating the value of a microbe:
* Ants - 1 vp per 2 microbes  
* Decomposers - 1 vp per 3 microbes  
* GHG Producing Bacteria - 1 temp per 2 microbes  
* Nitrite Reducing Bacteria - 1 TR per 3 microbes  
* Regolith Eaters - 1 oxygen per 2 microbes  
* Tardigrades - 1 vp per 4 microbes  

In [10]:
# microbe value
# microbe_tags = base_cards[base_cards.Tags.str.contains('microbe', na=False)]
# display(microbe_tags)

def microbe_val(generation, player_count):
    income, vp, TR = initialize(generation, player_count)
    
    ants = vp / 2
    decomposers = vp / 3
    ghg_producing_bacteria = temp / 2
    nitrite_reducing_bacteria = TR / 3
    regolith_eaters = greenery / 4
    tardigrades = vp / 4
    
    microbes_list = [ants, decomposers, ghg_producing_bacteria, nitrite_reducing_bacteria,
                     regolith_eaters, tardigrades]
    microbe = sum(microbes_list) / len(microbes_list)
    return microbe

vars_dict_resource.update({'microbe':microbe_val})

print(f'A microbe is worth {str(round(microbe_val(1, 2), 3))} credits in generation 1 with 2 players.')

A microbe is worth 3.905 credits in generation 1 with 2 players.


#### Animals

The following cards are taken into account when calculating the value of an animal:  
* Birds - 1 vp per animal
* Fish - 1 vp per animal
* Livestock - 1 vp per animal
* Predators - 1 vp per animal
* Ecological Zone - 1 vp per 2 animals
* Herbivores - 1 vp per 2 animals
* Pets - 1 vp per 2 animals
* Small Animals - 1 vp per 2 animals

In [11]:
# animal value
# animal_tags = base_cards[base_cards.Tags.str.contains('animal', na=False)]
# display(animal_tags)

def animal_val(generation, player_count):
    income, vp, TR = initialize(generation, player_count)
    
    birds= fish= livestock= predators = vp
    ecological_zone= herbivores= pets= small_animals = vp/2
    
    animals_list = [birds, fish, livestock, predators, ecological_zone,
                    herbivores, pets, small_animals]
    animal = sum(animals_list) / len(animals_list)
    return animal

vars_dict_resource.update({'animal':animal_val})

print(f'An animal is worth {str(round(animal_val(1, 2), 3))} credits in generation 1 with 2 players.')

An animal is worth 3.536 credits in generation 1 with 2 players.


#### Number of cities played

The average number of cities played in a game is set at 11. This is derived as a result of personal experience through dozens of games (mostly 2-player) as well as a rough averaging from the discussion [here](https://boardgamegeek.com/thread/1861021/how-many-cities).

There are two off-world city areas, and a couple of cards that ignore those cities. Because only 2 cards out of 208 allow a player to play on those areas, the same average value is used for cards that say "Cities" and those that say "Cities ON MARS".

The average number is cities is divided by the average number of generations to determine the number of cities on the board for any given generation and player count.

In [12]:
cities_per_game = 11
Mars_cities_per_game = 10

def cities_current(generation, player_count):
    for players in lastgens:
        if str(player_count) in players:
            lastgen = lastgens[players]
    cities = (11 / lastgen) * generation
    return cities

def Mars_cities_current(generation, player_count):
    for players in lastgens:
        if str(player_count) in players:
            lastgen = lastgens[players]
    Mars_cities = (10 / lastgen) * generation
    return Mars_cities

print(f'There are an average of {str(round(cities_current(6, 2)))} cities \
in generation 6 of a 2-player game.')
print(f'There are an average of {str(round(Mars_cities_current(8, 2)))} cities \
on Mars in generation 8 of a 2-player game.')

There are an average of 5 cities in generation 6 of a 2-player game.
There are an average of 6 cities on Mars in generation 8 of a 2-player game.


This brings me to the point of actually calculating the card values. My plan is to write a main function for doing this, then running it for most of the cards to create a values database.

Beyond that I've set aside fifteen active cards that have a cost to use. Ironworks, for example, costs 4 energy to use and grants a steel and raises the oxygen by 1. Do I calculate cards such as these as if they would be used in every remaining generation? That's probably what I'll do, I just haven't decided yet.

Finally there is a separate group of 23 cards that I don't know how to handle. An example would be Advanced Alloys, which increases the value of steel and titanium by one credit each. I'll probably have to have a separate function for each of those. Hopefully I'll be able to even figure out how to put numbers to each of them.

Before I start writing the probably very long function to determine the value of most of the cards, I will try a few card-specific functions just to get some practice and ideas about how to write the big one.

In [13]:
# Food Factory test
def food_factory(generation, player_count):
    income, vp, TR = initialize(generation, player_count)
    
    value = 0
    value -= 12 #card cost
    value -= round(plant*(income+1))
    value += round(4*income)
    value += vp
    display(f'The value of Food Factory in gen {str(generation)} with {str(player_count)} players is {str(round(value))} credits')
food_factory(1, 2)

'The value of Food Factory in gen 1 with 2 players is 4 credits'

In [14]:
# define special case cards to be left out of the main function
special_cases = ['Insulation', 'Ants', "CEO's Favorite Project", 'Land Claim', 'Lava Flows', 
                 'Robotic Workforce', 'Special Design', 'Adaptation Technology', 'Special Design'
                 'Nitrogen-Rich Asteroid', 'Nitrite Reducing Bacteria', 'Herbivores', 'Pets']

In [15]:
# Single card tester
Cartel = base_cards[base_cards.Title == 'Cartel']
display(Cartel)

,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent
33,Cartel,corporate,automated,8,NaN,earth,NaN,NaN,1 CREDIT per 1 player earth tag,NaN,NaN,NaN,NaN


In [16]:
# main value calculation function
def base_card_value(database, generation, player_count):
    income, vp, TR = initialize(generation, player_count)
    
    # iterate through cards in database
    for row in database.itertuples():
        dict_row = row._asdict()
        value = 0
        
        # Primary Cost
        value -= dict_row['Primary_Cost']
        
        # Additional Cost
        add_cost = dict_row['Additional_Cost']
        if type(add_cost) != float:
            add_cost_list_outer = add_cost.split(',')
            add_cost_list_inner = []
            for i in range(len(add_cost_list_outer)):
                add_cost_list_outer[i] = add_cost_list_outer[i].strip()
                add_cost_list_inner.append(add_cost_list_outer[i].split(' '))
            for i in range(len(add_cost_list_inner)):
                value -= vars_scan(add_cost_list_inner[i])
                            
        # Victory Points
        victory = dict_row['Victory_Points']
        if type(victory) != float:
            if len(victory) <= 2:
                value += int(victory) * vp        
                
        # Immediate Benefit
        i_benefit = dict_row['Immediate_Benefit']
        if type(i_benefit) != float:
            i_benefit_list_outer = i_benefit.split(',')
            i_benefit_list_inner = []
            for i in range(len(i_benefit_list_outer)):
                i_benefit_list_outer[i] = i_benefit_list_outer[i].strip()
                i_benefit_list_inner.append(i_benefit_list_outer[i].split(' '))
            for i in range(len(i_benefit_list_inner)):
                
                if 'per' in i_benefit_list_inner[i]:
                    value += pers(i_benefit_list_inner[i], generation, player_count)
                    
                elif 'or' in i_benefit_list_inner[i]:
                    or_benefit = (' ').join(i_benefit_list_inner[i])
                    value += ors(or_benefit, generation, player_count)
                    
                else:    # just keys from vars_dict (resources, standard projects, and TR)
                    value += vars_scan(i_benefit_list_inner[i])

        print(i_benefit_list_outer)
        print(i_benefit_list_inner)
        print(dict_row['Title'], round(value))
        print(initialize(11, 2))
            
base_card_value(Cartel, 11, 2)

['1 CREDIT per 1 player earth tag']
[['1', 'CREDIT', 'per', '1', 'player', 'earth', 'tag']]
Cartel 3
(2, 16.592592592592595, 18.592592592592595)


## General Notes -

* Ignoring opportunity cost. This is why a card draw is worth only 4. Certain cards will be devalued due to this, such as Business Contacts (where the extra two cards drawn and then discarded are not tallied)  
* Greeneries and Cities. The map point from a greenery is used in calculating the value of victory points, but it is not added to the value of the greenery standard project. Likewise, the potential map value of a city tile is not added to the value of the city standard project. The reasoning for this is that those map points/potential map points have been factored into determining the cost of the standard project.
* I chose in most cases to round only at the point of presenting the final card values.

#### Project Notes -
* In the `pers()` function in TM_lib.ipynb, the event portion is focused solely on Media Archives. Only "all events played" is handled. If I add any expansions later on, I will need to update this if any cards rely on the number of events played by the current player only.